# Using SVM do regression

## wiki [link](https://en.wikipedia.org/wiki/Support_vector_machine#Regression)

A version of SVM for regression was proposed in 1996 by Vladimir N. Vapnik, Harris Drucker, Christopher J. C. Burges, Linda Kaufman and Alexander J. Smola. This method is called support vector regression (SVR). The model produced by support vector classification (as described above) depends only on a subset of the training data, because the cost function for building the model does not care about training points that lie beyond the margin. Analogously, the model produced by SVR depends only on a subset of the training data, because the cost function for building the model ignores any training data close to the model prediction. Another SVM version known as least squares support vector machine (LS-SVM) has been proposed by Suykens and Vandewalle.

Training the original SVR means solving

minimize ${\displaystyle {\frac {1}{2}}\|w\|^{2}}$

subject to ${\displaystyle {\begin{cases}y_{i}-\langle w,x_{i}\rangle -b\leq \varepsilon \\\langle w,x_{i}\rangle +b-y_{i}\leq \varepsilon \end{cases}}} $
where ${\displaystyle x_{i}}$ is a training sample with target value ${\displaystyle y_{i}}$. The inner product plus intercept ${\displaystyle \langle w,x_{i}\rangle +b}$ is the prediction for that sample, and ${\displaystyle \varepsilon }$ is a free parameter that serves as a threshold: all predictions have to be within an ${\displaystyle \varepsilon }$ range of the true predictions. Slack variables are usually added into the above to allow for errors and to allow approximation in the case the above problem is infeasible.

<img src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Svr_epsilons_demo.svg">

$\varepsilon$越大，允许的误差范围就越大

- http://www.robots.ox.ac.uk/~az/lectures/ml/
- http://www.svms.org/regression/SmSc98.pdf
- http://svms.org/tutorials/Gunn1998.pdf
- https://cn.mathworks.com/help/stats/understanding-support-vector-machine-regression.html

## An Example
- http://scikit-learn.org/stable/auto_examples/svm/plot_svm_regression.html#sphx-glr-auto-examples-svm-plot-svm-regression-py

In [3]:
print(__doc__)

import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt

# #############################################################################
# Generate sample data
X = np.sort(5 * np.random.rand(40, 1), axis=0)
y = np.sin(X).ravel()

# #############################################################################
# Add noise to targets
y[::5] += 3 * (0.5 - np.random.rand(8))

# #############################################################################
# Fit regression model
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=1, epsilon=0.2)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=2)
y_rbf = svr_rbf.fit(X, y).predict(X)
y_lin = svr_lin.fit(X, y).predict(X)
y_poly = svr_poly.fit(X, y).predict(X)

# #############################################################################
# Look at the results
lw = 2
plt.scatter(X, y, color='darkorange', label='data')
plt.plot(X, y_rbf, color='navy', lw=lw, label='RBF model')
plt.plot(X, y_lin, color='c', lw=lw, label='Linear model')
plt.plot(X, y_poly, color='cornflowerblue', lw=lw, label='Polynomial model')
plt.xlabel('data')
plt.ylabel('target')
plt.title('Support Vector Regression')
plt.legend()
plt.savefig('example_SVR_4.png', dpi=200)
# plt.show()

Automatically created module for IPython interactive environment


In [3]:
y

array([ 1.26676326,  0.24908617,  0.3547268 ,  0.60508501,  0.80558322,
        0.73862292,  0.99971819,  0.99913522,  0.98216893,  0.96929906,
        0.67569163,  0.8080949 ,  0.64030547,  0.63822945,  0.60885156,
        0.90545958,  0.52432397,  0.50301796,  0.43150228,  0.28310435,
        1.08084544,  0.25088738, -0.02288449, -0.36260628, -0.49433258,
        0.5098226 , -0.58816107, -0.66524594, -0.67629795, -0.69038867,
       -1.95706922, -0.92627641, -0.937215  , -0.93743767, -0.95317115,
       -0.71257359, -0.98427415, -0.99108693, -0.99709302, -0.99462839])

### 用以前做线性回归的数据，尝试SCR，了解各个超参数的含义